<a href="https://colab.research.google.com/github/KimYongHee11/first/blob/main/%EB%B9%84%EC%A0%95%ED%98%95%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%B6%84%EC%84%9D_%EA%B9%80%EC%9A%A9%ED%9D%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install mxnet
!pip install gluonnlp
!pip install sentencepiece
!pip install transformers==3
!pip install torch
!pip install konlpy
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

     |████████████████████████████████| 46.9 MB 1.1 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 9.0 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595735 sha256=6c5fefe8d2ef54d41d19a46344b5522f6c7576ab2bdc0d97c818cff84859b01e
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 7.1 MB/s 
     |████████████████████████████████| 754 kB 8.8 MB/s 
     |████████████████████████████████| 895 kB 66.9 MB/s 
     |████████████████████████████████| 3.0 MB 72.2 MB/s 
     |████████████████████████████████| 19.4 MB 8.9 MB/s 
     |████████████████████████████████| 86 kB 7.6 MB/s 
     |████████████████████████████████| 448 kB 70.3 MB/s 
  Attempting unins

In [ ]:
import re
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

torch.manual_seed(615)
np.random.seed(615)

device = torch.device("cuda:0")
bertmodel, vocab = get_pytorch_kobert_model()

.cache/kobert_v1.zip[██████████████████████████████████████████████████]
.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [ ]:
import pandas as pd
import numpy as np
import re
import os
from konlpy.tag import Okt

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
matplotlib.rcParams['axes.unicode_minus'] = False
matplotlib.rc('font', family='Malgun Gothic')
%matplotlib inline

In [ ]:
import pandas as pd
path = '/content/drive/My Drive/비정형데이터 분석/페어링/'
train=pd.read_csv(path + 'train.csv')
test=pd.read_csv(path + 'test.csv')

In [ ]:
device = torch.device("cuda:0")

In [ ]:
train=train[['사업명', '과제명', '요약문_연구목표', '요약문_연구내용', '요약문_한글키워드', '요약문_기대효과', 'label']]
test=test[['사업명', '과제명', '요약문_연구목표', '요약문_연구내용', '요약문_기대효과', '요약문_한글키워드']]
train['요약문_연구목표'].fillna('NAN', inplace=True)
test['요약문_연구목표'].fillna('NAN', inplace=True)
train['요약문_연구내용'].fillna('NAN', inplace=True)
test['요약문_연구내용'].fillna('NAN', inplace=True)
train['요약문_기대효과'].fillna('NAN', inplace=True)
test['요약문_기대효과'].fillna('NAN', inplace=True)
train['요약문_한글키워드'].fillna('NAN', inplace=True)
test['요약문_한글키워드'].fillna('NAN', inplace=True)

train['data']= train['과제명'] 
test['data']=test['과제명'] 

def clean_text(sent):
    sent_clean=re.sub("[^가-힣ㄱ-하-ㅣ]", " ", sent)
    sent_clean = re.sub(' +', ' ', sent_clean) 
    return sent_clean


/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4536: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [ ]:
# 데이터 전처리
def make_preprocessed_data(data):
    train['과제명'].fillna('NAN', inplace=True)
    test['과제명'].fillna('NAN', inplace=True)

    train['과제명']= train['과제명'].str.lower()
    test['과제명']= test['과제명'].str.lower()


    train['과제명']= train['과제명'].str.replace(r'[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》●○#-]', ' ')
    test['과제명']= test['과제명'].str.replace(r'[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》●○#-]', ' ')

    train['과제명']= train['과제명'].str.replace('\n', ' ')
    test['과제명']= test['과제명'].str.replace('\n', ' ')
    

In [ ]:
train.apply(make_preprocessed_data)
test.apply(make_preprocessed_data)

사업명          None
과제명          None
요약문_연구목표     None
요약문_연구내용     None
요약문_기대효과     None
요약문_한글키워드    None
data         None
dtype: object

In [ ]:
def remove_special_characters(sentence, lower=True):
    p = re.compile('[\!@#$%\^&\*\(\)\-\=\[\]\{\}\.,/\?~\+\'"|_:;><`┃…]')
    s1 = re.compile('개발')
    s2 = re.compile('및')
    s3 = re.compile('위한')
    s4 = re.compile('기술')
    s5 = re.compile('연구')
    s6 = re.compile('기술개발')
    s8 = re.compile('기반')
    s9 = re.compile('시스템')
    s10 = re.compile('통한')
    s11 = re.compile('활용한')
    s12 = re.compile('대한')
    s13 = re.compile('분석')
    s14 = re.compile('관한')
    
    sentence = p.sub(' ', sentence) 
    sentence = s1.sub(' ', sentence)
    sentence = s2.sub(' ', sentence)
    sentence = s3.sub(' ', sentence)
    sentence = s4.sub(' ', sentence)
    sentence = s5.sub(' ', sentence)
    sentence = s6.sub(' ', sentence)
    sentence = s8.sub(' ', sentence)
    sentence = s9.sub(' ', sentence)
    sentence = s10.sub(' ', sentence)
    sentence = s11.sub(' ', sentence)
    sentence = s12.sub(' ', sentence)
    sentence = s13.sub(' ', sentence)
    sentence = s14.sub(' ', sentence)
    sentence = ' '.join(sentence.split()) 
    if lower:
        sentence = sentence.lower()
    return sentence

train['과제명'] =  train['과제명'].map(remove_special_characters)
test['과제명'] = test['과제명'].map(remove_special_characters) 

In [ ]:
from sklearn.model_selection import train_test_split
trains, tests = train_test_split(train, test_size=0.2, random_state=615)

In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

/root/kobert/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair) 

        self.sentences = [transform([clean_text(i)]) for i in dataset['data']]
        self.labels = [np.int32(i) for i in dataset['label']]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
max_len = 50
batch_size = 100
warmup_ratio = 0.1
num_epochs = 20
max_grad_norm = 1
log_interval = 150
learning_rate = 3e-5

In [ ]:
data_train = BERTDataset(trains, 3, 2, tok, max_len, True, False)
data_test = BERTDataset(tests, 3, 2, tok, max_len, True, False)

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=2)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=2)

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 46, 
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)


no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]


optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)


def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
highest_acc = 0
patience = 0

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm) 
        optimizer.step()
        scheduler.step() 
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

    
    model.eval()
    
    for test_batch_id, (test_token_ids, test_valid_length, test_segment_ids, test_label) in enumerate(tqdm_notebook(test_dataloader)):
        test_token_ids = test_token_ids.long().to(device)
        test_segment_ids = test_segment_ids.long().to(device)
        test_valid_length= test_valid_length
        test_label = test_label.long().to(device)
        test_out = model(token_ids, valid_length, segment_ids)
        test_loss = loss_fn(out, label)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (test_batch_id+1)))

    if test_acc > highest_acc:
        torch.save({
            'epoch': e,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': test_loss,
            }, '/content/drive/My Drive/비정형데이터 분석/페어링/model.pt')
        patience = 0
    else:
        print("test acc did not improved. best:{} current:{}".format(highest_acc, test_acc))
        patience += 1
        if patience > 5:
            break
    print('current patience: {}'.format(patience))
    print("************************************************************************************")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/1395 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 3.848989963531494 train acc 0.01
epoch 1 batch id 151 loss 2.8186452388763428 train acc 0.3090728476821193
epoch 1 batch id 301 loss 1.4787112474441528 train acc 0.5623920265780733
epoch 1 batch id 451 loss 1.1200159788131714 train acc 0.6460532150776057
epoch 1 batch id 601 loss 0.7134029865264893 train acc 0.688086522462562
epoch 1 batch id 751 loss 0.587333083152771 train acc 0.714354194407456
epoch 1 batch id 901 loss 1.0745975971221924 train acc 0.7317536071032177
epoch 1 batch id 1051 loss 0.972608745098114 train acc 0.7450333016175062
epoch 1 batch id 1201 loss 0.6854152679443359 train acc 0.7547876769358861
epoch 1 batch id 1351 loss 0.743703305721283 train acc 0.7634418948926711
epoch 1 train acc 0.765577394348585


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/349 [00:00<?, ?it/s]

...

epoch 19 test acc 1.0
current patience: 0
************************************************************************************


  0%|          | 0/1395 [00:00<?, ?it/s]

epoch 20 batch id 1 loss 0.016642697155475616 train acc 0.99
epoch 20 batch id 151 loss 0.02210223488509655 train acc 0.9896688741721852
epoch 20 batch id 301 loss 0.022433185949921608 train acc 0.9892026578073103
epoch 20 batch id 451 loss 0.04507191479206085 train acc 0.9893791574279414
epoch 20 batch id 601 loss 0.0031887025106698275 train acc 0.9892179700499217
epoch 20 batch id 751 loss 0.011486002244055271 train acc 0.9892010652463441
epoch 20 batch id 901 loss 0.06788305193185806 train acc 0.9892230854606059
epoch 20 batch id 1051 loss 0.01033664122223854 train acc 0.989438629876315
epoch 20 batch id 1201 loss 0.08591600507497787 train acc 0.9894088259783586
epoch 20 batch id 1351 loss 0.009475059807300568 train acc 0.9895114729829829
epoch 20 train acc 0.9894623655914053


  0%|          | 0/349 [00:00<?, ?it/s]

epoch 20 test acc 1.0
current patience: 0
************************************************************************************


In [ ]:
class BERTDataset1(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair) 

        self.sentences = [transform([clean_text(i)]) for i in dataset['data']]
        self.labels = [np.int32(0) for i in dataset['data']]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))
        #return (self.sentences[i] + (0, ))

    def __len__(self):
        return (len(self.sentences))


test_set = BERTDataset1(test, 3, 2, tok, max_len, True, False)
test_input = torch.utils.data.DataLoader(test_set, batch_size=1, num_workers=2)

In [ ]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

checkpoint = torch.load('/content/drive/My Drive/비정형데이터 분석/페어링/model.pt')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

In [ ]:
model.eval()


result = []

for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_input)):
  token_ids = token_ids.long().to(device)
  segment_ids = segment_ids.long().to(device)
  valid_length= valid_length
  label = label.long().to(device)
  out = model(token_ids, valid_length, segment_ids)
  for i in out:
      logits = i
      logits = logits.detach().cpu().numpy()
      final = np.argmax(logits)
  result.append(final)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/43576 [00:00<?, ?it/s]